# Data to Pandas table

**Purpose of script:**

Put mw and opt data to one pandas dataframe, create features. One row per pixel.

- In: opt and mw data, all files
- Out: one file (maybe several due to size constraints) (csv or parquet) with table of features prepared to be used in model

In [2]:
#import matplotlib.pyplot as plt
#import numpy as np

# import rioxarray
import xarray
# import rasterio
# import gdal

# from os import listdir
# from os.path import isfile, join

#%matplotlib inline

import pandas as pd
from datetime import datetime

Relevant paths:

In [3]:
mw_path = r"../Data/microwave-rs/mw_interpolated/2019-07-01_mw.tif"
opt_path = r"../Data/optical-rs/2019-07-01_grain_diameter.tif"

In [4]:
out_path = r"../Data/combined/"

Load data:

In [5]:
# load all files
# put to one xarray?

In [6]:
#TEMP Data load:
data_opt = xarray.open_dataarray(opt_path)
data_mw = xarray.open_dataarray(mw_path)

To pandas dataframe:

In [7]:
# for loop

In [8]:
# convert mw to pandas
df_mw = data_mw.to_dataframe()
df_opt = data_opt.to_dataframe()
# fix index
df_mw = df_mw.reset_index()
# remove columns: spacial_ref, band
df_mw = df_mw[['x', 'y', 'band_data']]
# rename
df_mw.rename({'band_data': 'mw_value'}, axis=1, inplace=True)
# ----------------------
# convert opt to pandas
df_opt = data_opt.to_dataframe()
# fix index
df_opt = df_opt.reset_index()
# remove columns: spacial_ref, band
df_opt = df_opt[['x', 'y', 'band_data']]
# rename
df_opt.rename({'band_data': 'opt_value'}, axis=1, inplace=True)
# ----------------------
# merge
df_comb = pd.merge(df_mw, df_opt, how = 'left', on = ['y', 'x']) # left smaller mw, right - opt
# fill na for masked opt data
df_comb.opt_value.fillna(-1, inplace=True)

In [9]:
# open_mfdataset() # baptiste 

In [10]:
# concat dim time (to put to ine file?)

Write to csv/ parquet: (?)

In [12]:
# write to csv
#df_comb.to_csv(out_path + 'melt_2019-07-01.csv', index= False)
df_comb.to_parquet(out_path + 'melt_2019-07-01.parquet.gzip', index= False)